In [1]:
!pip install cassandra-driver


  Obtaining dependency information for cassandra-driver from https://files.pythonhosted.org/packages/f8/26/adc5beac60c373733569868ee1843691fae5d9d8cd07a4907e7c4a55bdaa/cassandra_driver-3.29.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for geomet<0.3,>=0.1 from https://files.pythonhosted.org/packages/c9/81/156ca48f950f833ddc392f8e3677ca50a18cb9d5db38ccb4ecea55a9303f/geomet-0.2.1.post1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.1/364.1 kB 8.4 MB/s eta 0:00:00:00:01


In [2]:
pip install ace-tools

  Obtaining dependency information for ace-tools from https://files.pythonhosted.org/packages/27/c4/402d3ae2ecbfe72fbdcb2769f55580f1c54a3ca110c44e1efc034516a499/ace_tools-0.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


### Combining data for preprocessing

In [3]:
import pandas as pd

# Define file paths for existing CSVs
#base_path = "/Users/bhavikpatel/Desktop/RAG/RAG_Data/"  # Update to your directory
import pandas as pd

# Load the individual CSV files
instructions_df = pd.read_csv("/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/instructions.csv")
html_components_df = pd.read_csv("/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/html_components.csv")
css_styles_df = pd.read_csv("/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/css_styles.csv")

# Create a list to hold the unified rows
unified_data = []

# Add Instructions to the unified data
for _, row in instructions_df.iterrows():
    unified_data.append({
        "ID": f"Instruction-{row['instruction_id']}",
        "Source_Type": "Instruction",
        "Source_ID": row["instruction_id"],
        "Text_Representation": row["text"]
    })

# Add HTML Components to the unified data
for _, row in html_components_df.iterrows():
    text_representation = f"Name: {row['name']}, Attributes: {row['attributes']}, Context: {row['component_context']}."
    unified_data.append({
        "ID": f"HTML-{row['component_id']}",
        "Source_Type": "HTML_Component",
        "Source_ID": row["component_id"],
        "Text_Representation": text_representation
    })

# Add CSS Styles to the unified data
for _, row in css_styles_df.iterrows():
    text_representation = (f"Selector: {row['selector']}, Properties: {row['properties']}, "
                           f"Context: {row['description']}.")
    unified_data.append({
        "ID": f"CSS-{row['style_id']}",
        "Source_Type": "CSS_Style",
        "Source_ID": row["style_id"],
        "Text_Representation": text_representation
    })

# Convert the unified data to a DataFrame
unified_df = pd.DataFrame(unified_data)

# Save the unified DataFrame to a CSV file
output_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data.csv"
unified_df.to_csv(output_path, index=False)

# Display the first few rows of the dataframe to verify the content
print("Unified Data for Embedding:")
print(unified_df.head())

# Confirm the path of the saved file
print(f"Data saved to {output_path}")


Unified Data for Embedding:
              ID  Source_Type  Source_ID  \
0  Instruction-1  Instruction          1   
1  Instruction-2  Instruction          2   
2  Instruction-3  Instruction          3   
3  Instruction-4  Instruction          4   
4  Instruction-5  Instruction          5   

                                 Text_Representation  
0  Create a header of a personal portfolio websit...  
1  Create a header of a Photography Portfolio Web...  
2  Create a header for a Portfolio website under ...  
3  Create a header for a portfolio website in the...  
4  Create a header of a Personal Portfolio Websit...  
Data saved to /Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data.csv


In [4]:
import pandas as pd

# Load the unified CSV file
file_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data.csv"  # Update this path to the actual location of the file
try:
    unified_df = pd.read_csv(file_path)
    print("Unified CSV loaded successfully. Here's a preview:")
    display(unified_df.head())
    print("\nColumn Names:")
    print(unified_df.columns.tolist())
except Exception as e:
    print(f"Error loading file: {e}")

Unified CSV loaded successfully. Here's a preview:


,ID,Source_Type,Source_ID,Text_Representation
0,Instruction-1,Instruction,1,Create a header of a personal portfolio websit...
1,Instruction-2,Instruction,2,Create a header of a Photography Portfolio Web...
2,Instruction-3,Instruction,3,Create a header for a Portfolio website under ...
3,Instruction-4,Instruction,4,Create a header for a portfolio website in the...
4,Instruction-5,Instruction,5,Create a header of a Personal Portfolio Websit...



Column Names:
['ID', 'Source_Type', 'Source_ID', 'Text_Representation']


### Creating Embeddings

In [5]:
pip install sentence-transformers pandas


Note: you may need to restart the kernel to use updated packages.


In [6]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [7]:
# Path to the structured CSV file
file_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display a snippet of the data
print(data.head())

              ID  Source_Type  Source_ID  \
0  Instruction-1  Instruction          1   
1  Instruction-2  Instruction          2   
2  Instruction-3  Instruction          3   
3  Instruction-4  Instruction          4   
4  Instruction-5  Instruction          5   

                                 Text_Representation  
0  Create a header of a personal portfolio websit...  
1  Create a header of a Photography Portfolio Web...  
2  Create a header for a Portfolio website under ...  
3  Create a header for a portfolio website in the...  
4  Create a header of a Personal Portfolio Websit...  


### Loading SBERT Model

In [8]:
# Load the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Verify model is loaded
print("Model loaded successfully!")

Model loaded successfully!


### Generating Embeddings

In [9]:
# Generate embeddings for the 'Text_Representation' column
data['Embedding'] = data['Text_Representation'].apply(lambda x: model.encode(str(x)).tolist())

# Confirm embeddings are generated
print("Embeddings generated successfully!")


Embeddings generated successfully!


### Saving the file

In [13]:
# Save the DataFrame with embeddings to a new CSV file
output_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data_with_embeddings.csv"
data.to_csv(output_path, index=False)

print(f"Data with embeddings saved to: {output_path}")


Data with embeddings saved to: /Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data_with_embeddings.csv


### Testing the embeddings

In [11]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load the unified data file
file_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data_with_embeddings.csv"
data = pd.read_csv(file_path)

# Convert the 'Embedding' column from string to list
data['Embedding'] = data['Embedding'].apply(lambda x: np.array(eval(x)))

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define a sample query
query = "ecommerce Generate a responsive navigation bar with call to action button"

# Generate embedding for the query
query_embedding = model.encode(query).reshape(1, -1)

# Separate data by Source_Type
instructions = data[data['Source_Type'] == 'Instruction']
html_components = data[data['Source_Type'] == 'HTML_Component']
css_styles = data[data['Source_Type'] == 'CSS_Style']

# Compute cosine similarity for each source type
instructions['Similarity'] = cosine_similarity(query_embedding, np.vstack(instructions['Embedding'])).flatten()
html_components['Similarity'] = cosine_similarity(query_embedding, np.vstack(html_components['Embedding'])).flatten()
css_styles['Similarity'] = cosine_similarity(query_embedding, np.vstack(css_styles['Embedding'])).flatten()

# Weighted average of similarities for related components
weights = {'Instruction': 0.5, 'HTML_Component': 0.3, 'CSS_Style': 0.2}

# Initialize a dictionary to store combined similarity scores
combined_scores = {}

# Combine similarity scores by ID
for idx, row in instructions.iterrows():
    template_id = row['Source_ID']  # Assuming Source_ID links Instruction, HTML, and CSS
    instruction_score = row['Similarity']
    
    # Get related HTML and CSS scores
    html_score = html_components[html_components['Source_ID'] == template_id]['Similarity'].mean() if template_id in html_components['Source_ID'].values else 0
    css_score = css_styles[css_styles['Source_ID'] == template_id]['Similarity'].mean() if template_id in css_styles['Source_ID'].values else 0
    
    # Compute weighted average
    combined_score = (
        weights['Instruction'] * instruction_score +
        weights['HTML_Component'] * html_score +
        weights['CSS_Style'] * css_score
    )
    
    combined_scores[template_id] = combined_score

# Convert combined scores to a DataFrame for ranking
ranked_scores = pd.DataFrame.from_dict(combined_scores, orient='index', columns=['Combined_Similarity'])
ranked_scores = ranked_scores.sort_values(by='Combined_Similarity', ascending=False).reset_index()
ranked_scores.columns = ['Source_ID', 'Combined_Similarity']

# Display top 5 matches
print("Top 5 Matches:")
print(ranked_scores.head(5))


Top 5 Matches:
   Source_ID  Combined_Similarity
0        143             0.353227
1        230             0.352280
2        139             0.342654
3        192             0.340401
4        128             0.339958


/var/folders/r_/7p_x_zfn6m17765l8kqlmk9c0000gn/T/ipykernel_43908/1083413926.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instructions['Similarity'] = cosine_similarity(query_embedding, np.vstack(instructions['Embedding'])).flatten()
/var/folders/r_/7p_x_zfn6m17765l8kqlmk9c0000gn/T/ipykernel_43908/1083413926.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  html_components['Similarity'] = cosine_similarity(query_embedding, np.vstack(html_components['Embedding'])).flatten()
/var/folders/r_/7p_x_zfn

### Creating connection and loading files in chunks for vectorization in AstraDB

In [2]:
!pip install cassandra-driver


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
pip install --upgrade astrapy

  Obtaining dependency information for astrapy from https://files.pythonhosted.org/packages/db/48/684c270724bc3f8d12714556d201aa4610623da919505a6a09e56f50ef6a/astrapy-1.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for deprecation<2.2.0,>=2.1.0 from https://files.pythonhosted.org/packages/02/c3/253a89ee03fc9b9682f1541728eb66db7db22148cd94f89ab22528cd1e1b/deprecation-2.1.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pymongo>=3 from https://files.pythonhosted.org/packages/29/a2/9643450424bcf241e80bb713497ec2e3273c183d548b4eca357f75d71885/pymongo-4.10.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for uuid6>=2024.1.12 from https://files.pythonhosted.org/packages/d3/3e/4ae6af487ce5781ed71d5fe10aca72e7cbc4d4f45afc31b120287082a8dd/uuid6-2024.7.10-py3-none-any.whl.metadata
  Obtaining dependency information for h2<5,>=3 from https://files.pythonhosted.org/packages/2a/e5/db6d438da759efbb488c4f3fbdab7764492ff3c3f953132

### Connecting to AstraDB

In [2]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient("AstraCS:vjMNnOezJoNIynDjOyHvqOBA:bcd8c820a01ad41a6a456fc1cb31b679bdc2d2248dc1ec0ab1d66ddb3847cc64")
db = client.get_database_by_api_endpoint(
  "https://bb8c1ce4-1426-4fb4-9ee8-a9cfbe118864-us-east-2.apps.astra.datastax.com"
)

print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: []


### Creating new collection called 'embeddings'

In [5]:
# Create a collection named "embeddings"
if "embeddings" not in db.list_collection_names():
    db.create_collection("embeddings")
    print("Collection 'embeddings' created.")
else:
    print("Collection 'embeddings' already exists.")


Collection 'embeddings' created.


### Storing the generated embeddings inside the collection

In [8]:
import pandas as pd
import uuid

# Load your embeddings file
file_path = "/Users/smeetsheth/Documents/Data 298B/RAG Implemenation/CSV Files/unified_data_with_embeddings.csv"
data = pd.read_csv(file_path)

# Convert the 'Embedding' column from string to a Python list
data['Embedding'] = data['Embedding'].apply(lambda x: eval(x))

# Insert records into the collection
for _, row in data.iterrows():
    document = {
        "id": str(uuid.uuid4()),  # Unique identifier
        "source_id": row["ID"],  # Source ID (Instruction/HTML/CSS)
        "source_type": row["Source_Type"],  # Instruction, HTML, or CSS
        "text_representation": row["Text_Representation"],  # Text
        "embedding": row["Embedding"]  # Embedding as a list
    }
    db["embeddings"].insert_one(document)  # Corrected access to collection

print("All embeddings have been inserted into the 'embeddings' collection.")


All embeddings have been inserted into the 'embeddings' collection.


### Retrieving similar ID's based on the query text

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import ast

# Import DataAPIClient
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient("AstraCS:vjMNnOezJoNIynDjOyHvqOBA:bcd8c820a01ad41a6a456fc1cb31b679bdc2d2248dc1ec0ab1d66ddb3847cc64")
db = client.get_database_by_api_endpoint(
  "https://bb8c1ce4-1426-4fb4-9ee8-a9cfbe118864-us-east-2.apps.astra.datastax.com"
)

print(f"Connected to Astra DB: {db.list_collection_names()}")
# Define your collection name and get the collection
collection_name = "embeddings"
collection = db.get_collection(collection_name)

# Fetch all documents from the Astra DB collection
documents = list(collection.find({}))  # This returns all documents in the collection

# If embeddings are stored as strings, convert them to lists
for doc in documents:
    if isinstance(doc["embedding"], str):
        doc["embedding"] = ast.literal_eval(doc["embedding"])

# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Use a lightweight SBERT model

# Function to retrieve similar documents
def retrieve_similar_document(query_text, documents, top_k=3):
    try:
        # Embed the query
        query_embedding = sbert_model.encode([query_text])[0]

        # Extract embeddings and document IDs
        embeddings = np.array([doc["embedding"] for doc in documents])
        ids = [doc["id"] for doc in documents]
        texts = [doc["text_representation"] for doc in documents]

        # Compute cosine similarity
        similarities = cosine_similarity([query_embedding], embeddings).flatten()

        # Get top K most similar documents
        top_indices = similarities.argsort()[-top_k:][::-1]
        results = [{"id": ids[i], "similarity": float(similarities[i]), "text_representation": texts[i]} for i in top_indices]

        return results
    except Exception as e:
        print(f"Error during retrieval: {e}")
        return []

# Sample query text
query_text = "Design a modern header for a travel website with a minimalist layout."

# Retrieve top 5 similar documents
results = retrieve_similar_document(query_text, documents, top_k=5)

# Display the results
print("Top Similar Documents:")
for result in results:
    print(f"\nID: {result['id']}")
    print(f"Similarity: {result['similarity']:.2f}")
    print(f"Text Representation: {result['text_representation']}")


Connected to Astra DB: ['embeddings']
Top Similar Documents:

ID: 6e89f492-e870-4303-8096-d0bd3e9b74cb
Similarity: 0.75
Text Representation: Create a travel booking website header using HTML and CSS. The header should include a logo, navigation links (Visa, Supplier, API, Contact Us), a login and sign-up section, and a currency selector. Use Manrope font for modern and clean typography, and ensure the layout is responsive for all devices. The header should also include an eye-catching headline and subheadline, with call-to-action buttons and user ratings. Apply flexbox for easy alignment and spacing, emphasizing visual hierarchy and user-centered design principles.

ID: c4322a89-74b2-414c-9415-2c341eae8c1b
Similarity: 0.75
Text Representation: Create a travel booking website header using HTML and CSS. The header should include a logo, navigation links (Visa, Supplier, API, Contact Us), a login and sign-up section, and a currency selector. Use Manrope font for modern and clean typograph

### Function to retrieve relevant info

In [25]:
# New query text
query_text = "Create a footer section with social media icons and contact information."

# Call the retrieval function with the new query
new_results = retrieve_similar_document(query_text, documents, top_k=5)

# Print the new results
print("Top Similar Documents for new query:")
for result in new_results:
    print(f"\nID: {result['id']}")
    print(f"Similarity: {result['similarity']:.2f}")
    print(f"Text Representation: {result['text_representation']}")


Top Similar Documents for new query:

ID: fbeeb286-a4c0-4b7a-b3f7-0dd9aa9f0cf4
Similarity: 0.53
Text Representation: Create a header of a non-profit organization website with a clean, inclusive design that highlights social advocacy. Include a navigation bar with sections like Home, About, Services, Blogs, and Contact. Use visually engaging HTML and CSS components like transparent background overlays, hero images, call-to-action buttons, and accessible typography to enhance user experience and engagement.

ID: 83403101-2cd7-44ca-83a3-56de71ce5e76
Similarity: 0.52
Text Representation: Create a header of a Social Media Management Website showcasing a modern and vibrant interface. Utilize HTML for a structured layout, including navigation, banners, and call-to-action buttons. Implement CSS to ensure responsiveness, color gradients, animations, and bold typography. Prioritize accessibility, intuitive navigation, visual hierarchy, and clean UI components to enhance user engagement, embodyin